In [1]:
library(readxl)
library(tidyverse)
library(dplyr)
library(e1071)
library(xlsx)
library(rpart)
library(randomForest)
library(h2o)

Warning message:
"package 'readxl' was built under R version 3.6.1"Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'ggplot2' was built under R version 3.6.1"Warning message:
"package 'tibble' was built under R version 3.6.1"Warning message:
"package 'tidyr' was built under R version 3.6.1"Warning message:
"package 'readr' was built under R version 3.6.1"Warning message:
"package 'purrr' was built under R version 3.6.1"Warning message:
"package 'dplyr' was built under R version 3.6.1"Warning message:
"package 'stringr' was built under R version 3.6.1"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks s

In [2]:
train = read_excel("Data_Train.xlsx")
test = read_excel("Data_Test.xlsx")
test$Delivery_Time = NA
combi = rbind(train,test)

<center><h2>Data Exploration</h2></center>

In [3]:
combi

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",<U+20B9>200,<U+20B9>50,3.5,12,4,30 minutes
ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",<U+20B9>100,<U+20B9>50,3.5,11,4,30 minutes
ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",<U+20B9>150,<U+20B9>50,3.6,99,30,65 minutes
ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",<U+20B9>250,<U+20B9>99,3.7,176,95,30 minutes
ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",<U+20B9>200,<U+20B9>99,3.2,521,235,65 minutes
ID_5221,"Rmz Centennial, I Gate, Whitefield","South Indian, North Indian, Chinese",<U+20B9>150,<U+20B9>50,3.8,46,18,30 minutes
ID_3777,Mumbai Central,"Beverages, Fast Food",<U+20B9>150,<U+20B9>50,3.7,108,31,30 minutes
ID_745,Delhi University-GTB Nagar,"Chinese, Thai, Asian",<U+20B9>650,<U+20B9>50,4.0,1731,1235,45 minutes
ID_2970,Delhi University-GTB Nagar,"Mithai, Street Food",<U+20B9>100,<U+20B9>50,3.9,110,26,30 minutes
ID_3474,"Sector 1, Noida","Fast Food, North Indian, Rolls, Chinese, Momos, Mughlai",<U+20B9>200,<U+20B9>50,3.9,562,294,65 minutes


In [4]:
combi$Rating = as.numeric(combi$Rating)
combi$Votes = as.numeric(combi$Votes)
combi$Reviews = as.numeric(combi$Reviews)
sum(is.na(combi$Rating))
sum(is.na(combi$Votes))
sum(is.na(combi$Reviews))

Warning message in base::as.numeric(x):
"NAs introduced by coercion"Warning message in base::as.numeric(x):
"NAs introduced by coercion"Warning message in base::as.numeric(x):
"NAs introduced by coercion"

[1] 2470

[1] 2616

[1] 2905

In [5]:
combi$Delivery_Time = gsub(" [a-z]*","",combi$Delivery_Time)
combi$Delivery_Time = as.numeric(combi$Delivery_Time)
sum(is.na(combi$Delivery_Time))

[1] 2774

In [6]:
combi$Restaurant = gsub("ID_","",combi$Restaurant)
combi$Restaurant = as.factor(combi$Restaurant)
summary(combi$Restaurant)

5538    7184    2483    1209    4654    7295    5321    4625    1140    2301 
     26      25      24      23      23      23      22      21      20      20 
   7750    8511    5708     188    3970    4906    7564    3969     694     720 
     20      19      18      17      17      17      17      16      16      16 
   7544    2542    2402    6229    7260    5952    2794    4652    5885     686 
     16      15      14      14      14      13      12      12      12      12 
   7501    1015    1657    3116    3203    4446    4562    8495    8606    1487 
     12      11      11      11      11      11      11      11      11      10 
   1635    3629     463    6163    6899     847    8605    1156    2356    2733 
     10      10      10      10      10      10      10       9       9       9 
   2960    3272    4216    5903    7142    7654    8170    8418    1784    2907 
      9       9       9       9       9       9       9       9       8       8 
   3871    4371    4839    5084    6205    6637     745    8438    1074    1146 
      8       8       8       8       8       8       8       8       7       7 
   1454    1528    1911    2905    4468    4922    5002    5093    5111    5252 
      7       7       7       7       7       7       7       7       7       7 
   5400    6054    6366    7200    7314    8026    8405     849    1036    1042 
      7       7       7       7       7       7       7       7       6       6 
   1444     164    1792    2326     244    2660    2678    2772    2781 (Other) 
      6       6       6       6       6       6       6       6       6   12769

In [7]:
head(combi)

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",<U+20B9>200,<U+20B9>50,3.5,12,4,30
2882,"Sector 3, Marathalli","Ice Cream, Desserts",<U+20B9>100,<U+20B9>50,3.5,11,4,30
1595,Mumbai Central,"Italian, Street Food, Fast Food",<U+20B9>150,<U+20B9>50,3.6,99,30,65
5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",<U+20B9>250,<U+20B9>99,3.7,176,95,30
6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",<U+20B9>200,<U+20B9>99,3.2,521,235,65
5221,"Rmz Centennial, I Gate, Whitefield","South Indian, North Indian, Chinese",<U+20B9>150,<U+20B9>50,3.8,46,18,30


In [8]:
combi$Average_Cost = str_extract(combi$Average_Cost,"[0-9]+")
combi$Average_Cost = as.numeric(combi$Average_Cost)
sum(is.na(combi$Average_Cost))

[1] 1

In [9]:
head(combi)

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,<U+20B9>50,3.5,12,4,30
2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,<U+20B9>50,3.5,11,4,30
1595,Mumbai Central,"Italian, Street Food, Fast Food",150,<U+20B9>50,3.6,99,30,65
5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,<U+20B9>99,3.7,176,95,30
6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,<U+20B9>99,3.2,521,235,65
5221,"Rmz Centennial, I Gate, Whitefield","South Indian, North Indian, Chinese",150,<U+20B9>50,3.8,46,18,30


<center><h2>Missing Value Treatment</h2></center>

In [10]:
combi$Minimum_Order = str_extract(combi$Minimum_Order,"[0-9]+")
combi$Minimum_Order = as.numeric(combi$Minimum_Order)
sum(is.na(combi$Minimum_Order))

[1] 0

In [11]:
combi$Location = as.factor(combi$Location)

In [12]:
summary(combi$Location)

Babarpur, New Delhi, Delhi 
                                             337 
                   BTM Layout 1, Electronic City 
                                              30 
                          Chandni Chowk, Kolkata 
                                             156 
              Chatta Bazaar, Malakpet, Hyderabad 
                                              67 
                       D-Block, Sector 63, Noida 
                                            1027 
            Delhi Administration Flats, Timarpur 
                                             602 
                                    Delhi Cantt. 
                                             397 
                    Delhi High Court, India Gate 
                                             504 
                      Delhi University-GTB Nagar 
                                             699 
                  Dockyard Road, Mumbai CST Area 
                                             362 
             FTI College, Law College Road, Pune 
                                             734 
      Gora Bazar, Rajbari, North Dumdum, Kolkata 
                                              95 
               Hyderabad Public School, Begumpet 
                                              93 
                 Jaya Nagar, Saidabad, Hyderabad 
                                              61 
Laxman Vihar Industrial Area, Sector 3A, Gurgoan 
                                             611 
                                        Majestic 
                                             411 
                                   MG Road, Pune 
                                             509 
      Mico Layout, Stage 2, BTM Layout,Bangalore 
                                            1158 
                                Moulali, Kolkata 
                                             173 
                                  Mumbai Central 
                                             578 
                 Musi Nagar, Malakpet, Hyderabad 
                                              91 
               Nathan Road, Mangaldas Road, Pune 
                                             476 
            Noorkhan Bazaar, Malakpet, Hyderabad 
                                              79 
            Panjetan Colony, Malakpet, Hyderabad 
                                              73 
                                 Pune University 
                                             344 
                             Raja Bazar, Kolkata 
                                             138 
              Rmz Centennial, I Gate, Whitefield 
                                             585 
                 Sandhurst Road, Mumbai CST Area 
                                             463 
                                 Sector 1, Noida 
                                             990 
                                Sector 14, Noida 
                                             630 
                            Sector 3, Marathalli 
                                             444 
                              Sector 63A,Gurgaon 
                                             323 
         Tejas Nagar Colony, Wadala West, Mumbai 
                                             133 
                                Tiretti, Kolkata 
                                             116 
                     Yerawada, Pune, Maharashtra 
                                             379

In [13]:
a = combi$Location

In [14]:
combi$Location = as.integer(combi$Location)

In [15]:
combi$Location = as.factor(combi$Location)

In [16]:
summary(combi$Location)

1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16 
 337   30  156   67 1027  602  397  504  699  362  734   95   93   61  611  411 
  17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32 
 509 1158  173  578   91  476   79   73  344  138  585  463  990  630  444  323 
  33   34   35 
 133  116  379

In [17]:
missing_index = which(is.na(combi$Rating))
for(i in missing_index){
    item = combi$Rating[i]
    combi$Rating[i] = mean(combi$Rating, na.rm = T)
}

In [18]:
sum(is.na(combi$Rating))

[1] 0

In [19]:
missing_index = which(is.na(combi$Reviews))
for(i in missing_index){
    item = combi$Reviews[i]
    combi$Reviews[i] = mean(combi$Reviews, na.rm = T)
}

In [20]:
sum(is.na(combi$Reviews))

[1] 0

In [21]:
missing_index = which(is.na(combi$Votes))
for(i in missing_index){
    item = combi$Votes[i]
    combi$Votes[i] = 0
}

In [22]:
sum(is.na(combi$Votes))

[1] 0

In [23]:
missing_index = which(is.na(combi$Average_Cost))
for(i in missing_index){
    item = combi$Average_Cost[i]
    combi$Average_Cost[i] = mean(combi$Average_Cost, na.rm = T)
}

In [24]:
sum(is.na(combi$Average_Cost))

[1] 0

In [25]:
dim(combi)

[1] 13868     9

In [26]:
head(combi)

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
6321,11,"Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12,4,30
2882,31,"Ice Cream, Desserts",100,50,3.5,11,4,30
1595,20,"Italian, Street Food, Fast Food",150,50,3.6,99,30,65
5929,29,"Mughlai, North Indian, Chinese",250,99,3.7,176,95,30
6123,27,"Cafe, Beverages",200,99,3.2,521,235,65
5221,27,"South Indian, North Indian, Chinese",150,50,3.8,46,18,30


In [27]:
str(combi)

Classes 'tbl_df', 'tbl' and 'data.frame':	13868 obs. of  9 variables:
 $ Restaurant   : Factor w/ 8661 levels "0","1","10","100",..: 5916 2094 664 5479 5696 4694 3088 7169 2192 2752 ...
 $ Location     : Factor w/ 35 levels "1","2","3","4",..: 11 31 20 29 27 27 20 9 9 29 ...
 $ Cuisines     : chr  "Fast Food, Rolls, Burger, Salad, Wraps" "Ice Cream, Desserts" "Italian, Street Food, Fast Food" "Mughlai, North Indian, Chinese" ...
 $ Average_Cost : num  200 100 150 250 200 150 150 650 100 200 ...
 $ Minimum_Order: num  50 50 50 99 99 50 50 50 50 50 ...
 $ Rating       : num  3.5 3.5 3.6 3.7 3.2 3.8 3.7 4 3.9 3.9 ...
 $ Votes        : num  12 11 99 176 521 ...
 $ Reviews      : num  4 4 30 95 235 ...
 $ Delivery_Time: num  30 30 65 30 65 30 30 45 30 65 ...


In [28]:
combi[,4:8] = scale(combi[,4:8])

In [29]:
train = combi[1:nrow(train),]
test = combi[(nrow(train) + 1):nrow(combi),]
test$Delivery_Time = NULL

In [30]:
head(train)

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
6321,11,"Fast Food, Rolls, Burger, Salad, Wraps",0.009912704,-0.1796816,-0.29555387,-0.36429646,-0.41658990,30
2882,31,"Ice Cream, Desserts",-0.827284864,-0.1796816,-0.29555387,-0.36628087,-0.41658990,30
1595,20,"Italian, Street Food, Fast Food",-0.408686080,-0.1796816,-0.02878771,-0.19165294,-0.32392963,65
5929,29,"Mughlai, North Indian, Chinese",0.428511488,2.4597911,0.23797844,-0.03885351,-0.09227895,30
6123,27,"Cafe, Beverages",0.009912704,2.4597911,-1.09585233,0.64576734,0.40666097,65
5221,27,"South Indian, North Indian, Chinese",-0.408686080,-0.1796816,0.50474459,-0.29682658,-0.36669591,30


In [31]:
head(test)

Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
2842,18,"North Indian, Chinese, Assamese",1.265709057,-0.1796816,1.57180921,0.3282620,0.3710224
730,18,"Biryani, Kebab",-0.827284864,-0.1796816,0.00000000,-0.3881094,0.0000000
4620,29,Fast Food,-0.827284864,-0.1796816,-0.02878771,-0.3166707,-0.3738236
5470,1,"Mithai, North Indian, Chinese, Fast Food, South Indian",0.009912704,-0.1796816,-0.02878771,-0.2571384,-0.3132381
3249,29,"Chinese, Fast Food",-0.408686080,-0.1796816,-1.89615079,-0.3127018,-0.3809513
506,35,"North Indian, Chinese",-0.827284864,-0.1796816,-0.56232002,-0.3563588,-0.3916429


In [32]:
dim(test)

[1] 2774    8

In [33]:
dim(train)

[1] 11094     9

<center><h2>Support vector Regressor</h2></center>

In [34]:
regressor = svm(formula = Delivery_Time ~ .,
                data = train[,-3],
                type = 'nu-regression',
                kernel = 'linear')

In [35]:
regressor


Call:
svm(formula = Delivery_Time ~ ., data = train[, -3], type = "nu-regression", 
    kernel = "linear")


Parameters:
   SVM-Type:  nu-regression 
 SVM-Kernel:  linear 
       cost:  1 
         nu:  0.5 

Number of Support Vectors:  10789


In [36]:
# Predicting a new result
y_pred = predict(regressor, test[,-3])

In [37]:
mround <- function(x,base){
        base*round(x/base)
}

In [38]:
b = mround(y_pred,5)

In [39]:
b = paste(b,"minutes",sep = ' ')

In [40]:
head(b)

[1] "35 minutes" "30 minutes" "30 minutes" "35 minutes" "30 minutes"
[6] "30 minutes"

In [41]:
write.xlsx(b,file = "svm.xlsx")

In [42]:
dim(b)

NULL

In [43]:
length(y_pred)

[1] 2774

In [44]:
dim(train)

[1] 11094     9

In [45]:
dim(test[,-3])

[1] 2774    7

In [46]:
summary(is.na(test))

 Restaurant       Location        Cuisines       Average_Cost   
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2774      FALSE:2774      FALSE:2774      FALSE:2774     
 Minimum_Order     Rating          Votes          Reviews       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:2774      FALSE:2774      FALSE:2774      FALSE:2774     

<center><h2>Decision Tree</h2></center>

In [47]:
regressor = rpart(formula = Delivery_Time ~ .,
                  data = train[,-3],
                  control = rpart.control(minsplit = 4))

# Predicting a new result with Decision Tree Regression
y_pred = predict(regressor, test[,-3])

In [48]:
b = mround(y_pred,5)

In [49]:
b = paste(b,"minutes",sep = " ")

In [50]:
head(b)

[1] "45 minutes" "30 minutes" "30 minutes" "30 minutes" "30 minutes"
[6] "30 minutes"

In [51]:
write.xlsx(b,file = "decision.xlsx")

In [52]:
length(b)

[1] 2774

In [53]:
tail(b)

[1] "45 minutes" "30 minutes" "45 minutes" "45 minutes" "30 minutes"
[6] "65 minutes"

<center><h2>Random Forest</h2></center>

In [54]:
regressor = randomForest(train[,-c(1,3,9)],train$Delivery_Time,ntree = 1000)

In [55]:
y_pred = predict(regressor, test[,-3])

In [56]:
b = mround(y_pred,5)

In [57]:
b = paste(b,"minutes",sep = " ")

In [58]:
head(b)

[1] "35 minutes" "30 minutes" "40 minutes" "35 minutes" "40 minutes"
[6] "30 minutes"

In [59]:
write.xlsx(b,file = "random.xlsx")

<center><h2>Artificial Neural Network</h2></center>

In [60]:
h2o.init(nthreads = -1)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    C:\Users\Dell\AppData\Local\Temp\RtmpSW0xyu/h2o_Dell_started_from_r.out
    C:\Users\Dell\AppData\Local\Temp\RtmpSW0xyu/h2o_Dell_started_from_r.err


Starting H2O JVM and connecting: .. Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         9 seconds 659 milliseconds 
    H2O cluster timezone:       Asia/Kolkata 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.26.0.2 
    H2O cluster version age:    5 months and 8 days !!! 
    H2O cluster name:           H2O_started_from_R_Dell_tcf657 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   1.98 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 


Warning message in h2o.clusterInfo():
"
Your H2O cluster version is too old (5 months and 8 days)!
Please download and install the latest version from http://h2o.ai/download/"

In [61]:
regressor = h2o.deeplearning(y = "Delivery_Time",
                              training_frame = as.h2o(train[,-3]),
                              activation = "TanhWithDropout",
                             hidden = c(50,50),
                              epochs = 100,
                              train_samples_per_iteration = -2,
                            keep_cross_validation_models = TRUE)

  |======================================================================| 100%
  |======================================================================| 100%


In [62]:
regressor

Model Details:

H2ORegressionModel: deeplearning
Model ID:  DeepLearning_model_R_1578139888977_1 
Status of Neuron Layers: predicting Delivery_Time, regression, gaussian distribution, Quadratic loss, 378,751 weights/biases, 4.5 MB, 1,018,699 training samples, mini-batch size 1
  layer units        type dropout       l1       l2 mean_rate rate_rms momentum
1     1  7522       Input  0.00 %       NA       NA        NA       NA       NA
2     2    50 TanhDropout 50.00 % 0.000000 0.000000  0.005052 0.016253 0.000000
3     3    50 TanhDropout 50.00 % 0.000000 0.000000  0.009528 0.012552 0.000000
4     4     1      Linear      NA 0.000000 0.000000  0.000208 0.000010 0.000000
  mean_weight weight_rms mean_bias  bias_rms
1          NA         NA        NA        NA
2   -0.003877   0.439310  2.774671 32.783051
3   -0.045366   2.491865 -0.266127  2.716818
4   -0.003233   0.055294  0.362789  0.000000


H2ORegressionMetrics: deeplearning
** Reported on training data. **
** Metrics reported on temp

In [63]:
y_pred = h2o.predict(regressor,newdata = as.h2o(test[-3]))

  |======================================================================| 100%
  |======================================================================| 100%


Warning message in doTryCatch(return(expr), name, parentenv, handler):
"Test/Validation dataset column 'Restaurant' has levels not trained on: [10, 1002, 1024, 1030, 1040, 1045, 106, 1060, 1066, 1075, 1087, 1097, 1111, 1113, 1117, 1129, 113, 1132, 1141, 1147, 115, 1151, 1154, 116, 1161, 1176, 1178, 1186, 1187, 1188, 1196, 1201, 1202, 1206, 1211, 1221, 1229, 123, 1259, 127, 1280, 1295, 130, 1307, 1314, 1320, 1323, 1326, 1330, 1337, 1351, 1354, 1356, 1357, 1369, 137, 1386, 1387, 1395, 1409, 1410, 1412, 1418, 1433, 1437, 1438, 144, 1442, 1445, 1473, 1474, 1491, 1498, 1500, 1502, 1509, 1512, 1515, 1518, 1525, 1534, 1536, 154, 1542, 1567, 157, 1572, 1573, 1576, 1583, 1587, 1591, 1597, 16, 1601, 161, 1624, 1629, 1630, 1632, 1634, 1649, 1652, 1660, 167, 1674, 1689, 169, 1691, 1702, 1707, 1708, 171, 1712, 1715, 1722, 173, 1747, 1748, 1750, 1764, 1768, 1779, 1790, 1814, 1816, 185, 1851, 1866, 1873, 1875, 1880, 1888, 1898, 1903, 1921, 194, 1941, 1944, 1950, 1963, 1970, 1973, 1977, 1982, 1983, 19

In [64]:
y_pred

   predict
1 43.87471
2 31.77999
3 32.28231
4 31.71662
5 32.36931
6 31.91659

[2774 rows x 1 column] 

In [65]:
b = mround(y_pred,5)

In [66]:
b = as.vector(b)

In [67]:
b = paste(b,"minutes",sep = " ")

In [68]:
write.xlsx(b,file = "ann.xlsx")